# Predeict the heatwave

In [2]:
import pandas as pd

In [3]:
#import data
url="https://github.com/Themaoyc/MDA/blob/main/Data/deaths_predict.csv?raw=true"
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Year,Country,ISO,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Dis Mag Value,duration
0,0,2000,Bulgaria,BGR,7.0,63.549547,"28,124.88",8170172.0,94.497452,34.600000,34,31
1,1,2000,Cyprus,CYP,5.0,63.549547,"10,594.98",943288.0,750.516724,34.100000,34,1
2,2,2000,Greece,GRC,27.0,63.549547,"141,247.28",10805808.0,885.410400,33.800000,33,1
3,7,2000,Turkey,TUR,11.0,63.549547,"171,494.21",63240196.0,199.499374,27.600000,27,31
4,8,2000,Pakistan,PAK,24.0,63.549547,...,142343583.0,15.581897,29.235238,29,1
